In [ ]:
import sys
from pathlib import Path

import pandas as pd
import yaml
from IPython.display import display

sys.path.insert(0, "..")
from runner.utils import (
    allocate_benchmarks,
    create_benchmark_campaign,
    load_benchmark_metadata,
)

In [2]:
# If a util function was modified, use this cell to reload it without having to restart the kernel
%run ../runner/utils.py

## Setup benchmark campaign

### 20251128 Test HiPO on paper's PyPSA instances

In [4]:
urls = """https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-op-10-12h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-op-10-1h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-op-10-24h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-op-10-3h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-op-2-12h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-op-2-1h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-op-2-24h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-op-2-3h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-op-3-12h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-op-3-1h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-op-3-24h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-op-3-3h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-op-4-12h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-op-4-1h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-op-4-24h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-op-4-3h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-op-5-12h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-op-5-1h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-op-5-24h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-op-5-3h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-op-6-12h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-op-6-1h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-op-6-24h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-op-6-3h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-op-7-12h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-op-7-1h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-op-7-24h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-op-7-3h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-op-8-12h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-op-8-1h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-op-8-24h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-op-8-3h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-op-9-12h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-op-9-1h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-op-9-24h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-op-9-3h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-trex-10-12h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-trex-10-1h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-trex-10-24h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-trex-10-3h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-trex-2-12h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-trex-2-1h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-trex-2-24h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-trex-2-3h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-trex-3-12h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-trex-3-1h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-trex-3-24h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-trex-3-3h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-trex-4-12h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-trex-4-1h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-trex-4-24h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-trex-4-3h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-trex-5-12h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-trex-5-1h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-trex-5-24h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-trex-5-3h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-trex-6-12h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-trex-6-1h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-trex-6-24h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-trex-6-3h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-trex-7-12h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-trex-7-1h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-trex-7-24h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-trex-7-3h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-trex-8-12h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-trex-8-1h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-trex-8-24h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-trex-8-3h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-trex-9-12h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-trex-9-1h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-trex-9-24h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-trex-9-3h.lp"""

benchmarks = {}
for url in urls.strip().split("\n"):
    # Extract filename from URL
    filename = url.split("/")[-1].replace(".lp", "")
    # Parse benchmark name and size (first 4 parts as benchmark name, rest as size name)
    parts = filename.split("-")
    if len(parts) >= 5:
        key = "-".join(parts[:4])
        name = "-".join(parts[4:])
        if key not in benchmarks:
            benchmarks[filename] = {"Sizes": []}
        benchmarks[filename]["Sizes"].append({"Name": name, "Size": None, "URL": url})

# Write to yaml file
with open("../pypsa-hipo-benchmarks.yaml", "w") as f:
    yaml.dump({"benchmarks": benchmarks}, f, default_flow_style=False, sort_keys=False)

In [5]:
with open("../pypsa-hipo-benchmarks.yaml", "r") as f:
    hipo_benchmarks = yaml.safe_load(f)

In [6]:
def get_machine_type(size):
    return "c4-highmem-8" if size == "L" else "c4-standard-2"

In [7]:
# Create hipo benchmarks

# Create output directory for HIPO benchmarks
hipo_output_dir = Path("../infrastructure/benchmarks/20251128-test-hipo")
hipo_output_dir.mkdir(parents=True, exist_ok=True)

# Check for existing yaml files and clean them up
existing_files = list(hipo_output_dir.glob("*.yaml"))
if existing_files:
    print(
        f"Warning: Found {len(existing_files)} existing yaml files in {hipo_output_dir}"
    )
    for f in existing_files:
        f.unlink()
    print("Cleaned up existing files.")

# Pick the largest 4 instances of each bench
# Filter pypsa-eur-elec-op
op_df = hipo_benchmarks[
    hipo_benchmarks["Benchmark"].str.startswith("pypsa-eur-elec-op")
]
op_df = op_df.sort_values("Num. variables").tail(4)

# Convert to list of (name, record) pairs
op_benchs = [(row.Benchmark, row) for _, row in op_df.iterrows()]

# Same for trex
tr_df = hipo_benchmarks[
    hipo_benchmarks["Benchmark"].str.startswith("pypsa-eur-elec-trex")
]
tr_df = tr_df.sort_values("Num. variables").tail(4)
tr_benchs = [(row.Benchmark, row) for _, row in tr_df.iterrows()]

# Generate YAML files
hipo_idx = 0
for benchmark_name, row in op_benchs + tr_benchs:
    # Remove size suffix from benchmark name
    benchmark_base = "-".join(benchmark_name.split("-")[:-2])

    size_data = {"Name": row.Instance, "Size": row["Size"], "URL": row["URL"]}

    output_filename = f"{hipo_idx:02d}-pypsa-hipo.yaml"
    output_path = hipo_output_dir / output_filename

    output_yaml = {
        "machine-type": get_machine_type(row["Size"]),
        "zone": "us-central1-a",
        "years": [2025],
        "solver": "highs-hipo-ipm highs-hipo-64 highs-hipo-no2hop",
        "benchmarks": {benchmark_base: {"Sizes": [size_data]}},
    }

    with open(output_path, "w") as f:
        yaml.dump(output_yaml, f, default_flow_style=False, sort_keys=False)

    print(f"Created: {output_filename}")
    hipo_idx += 1

print(f"\nTotal HIPO files created: {len(list(hipo_output_dir.glob('*.yaml')))}")

KeyError: 'Benchmark'

In [8]:
with open("../pypsa-hipo-benchmarks.yaml", "r") as f:
    hipo_benchmarks = yaml.safe_load(f)

### 20251201 Test new large PyPSA-Eur instances

In [9]:
# Load the new benchmarks
with open("../benchmarks/pypsa/metadata.yaml", "r") as f:
    new_benchmarks = yaml.safe_load(f)

In [10]:
# Create benchmark campaign
output_dir = Path("../infrastructure/benchmarks/20251201-hipo-new-pypsa")
output_dir.mkdir(parents=True, exist_ok=True)


# Determine machine type and solvers based on size and problem class
def get_machine_type(size):
    return "c4-highmem-8" if size == "L" else "c4-standard-2"


idx = 0
for benchmark_name, benchmark_data in new_benchmarks["benchmarks"].items():
    for size_data in benchmark_data.get("Sizes", []):
        size = size_data["Size"]

        output_filename = f"{idx:02d}-hipo-new-pypsa.yaml"
        output_path = output_dir / output_filename
        output_yaml = {
            "machine-type": get_machine_type(size),
            "zone": "us-central1-a",
            "years": [2025],
            "solver": "gurobi highs-hipo-no2hop highs-hipo-64 highs-hipo-ipm",
            "timeout_seconds": 24 * 60 * 60,
            "benchmarks": {
                benchmark_name: {
                    "Sizes": [
                        {
                            "Name": size_data["Name"],
                            "Size": size,
                            "URL": size_data["URL"],
                        }
                    ]
                }
            },
        }

        with open(output_path, "w") as f:
            yaml.dump(output_yaml, f, default_flow_style=False, sort_keys=False)
        # print(f"Created: {output_filename}")
        idx += 1

print(f"\nTotal files created: {len(list(output_dir.glob('*.yaml')))}")


Total files created: 222


In [ ]:
rows = []
for bench, meta in new_benchmarks["benchmarks"].items():
    for size in meta["Sizes"]:
        rows.append(
            {
                "Benchmark": f"{bench}-{size['Name']}",
                "Num. variables": size["Num. variables"],
            }
        )

df = pd.DataFrame(rows).sort_values("Num. variables")

display(df.style.hide(axis="index").format({"Num. variables": "{:,.0f}"}))

In [12]:
# Check the results manually (can't use utils.load_results because this run didn't record `VM Zone1`)
csv_files = [
    p for p in Path("../results/gcp-results/20251201-hipo-new-pypsa/").glob("*.csv")
]
results = pd.concat([pd.read_csv(p) for p in csv_files]).reset_index(drop=True)
reference_results = results.query('Benchmark == "reference-benchmark"')
results = results.query('Benchmark != "reference-benchmark"').copy()
results

/tmp/ipykernel_61490/3486318512.py:5: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results = pd.concat([pd.read_csv(p) for p in csv_files]).reset_index(drop=True)


,Benchmark,Size,Solver,Solver Version,Solver Release Year,Status,Termination Condition,Runtime (s),Memory Usage (MB),Objective Value,Max Integrality Violation,Duality Gap,Reported Runtime (s),Timeout,Hostname,Run ID,Timestamp
0,pypsa-eur-elec-trex_vopt,50-1h,gurobi,12.0.3,2025.0,TO,Timeout,86400.000000,30807.732,NaN,NaN,NaN,86400.000000,86400.0,benchmark-instance-04-hipo-new-pypsa,20251201-hipo-new-pypsa,2025-12-01 08:19:33.382964
2,pypsa-eur-sec-trex_vopt,50-1h,gurobi,12.0.3,2025.0,OOM,Out of Memory,NaN,NaN,NaN,NaN,NaN,NaN,86400.0,benchmark-instance-00-hipo-new-pypsa,20251201-hipo-new-pypsa,2025-12-01 08:22:49.404869
4,pypsa-eur-sec-trex_vopt,50-1h,highs-hipo-no2hop,1.12.0-hipo,2025.0,OOM,Out of Memory,NaN,NaN,NaN,NaN,NaN,NaN,86400.0,benchmark-instance-00-hipo-new-pypsa,20251201-hipo-new-pypsa,2025-12-01 08:32:01.479494
5,pypsa-eur-sec-trex_vopt,50-1h,highs-hipo-64,1.12.0-hipo,2025.0,OOM,Out of Memory,NaN,NaN,NaN,NaN,NaN,NaN,86400.0,benchmark-instance-00-hipo-new-pypsa,20251201-hipo-new-pypsa,2025-12-01 08:35:51.116007
6,pypsa-eur-sec-trex_vopt,50-1h,highs-hipo-ipm,1.12.0-hipo,2025.0,OOM,Out of Memory,NaN,NaN,NaN,NaN,NaN,NaN,86400.0,benchmark-instance-00-hipo-new-pypsa,20251201-hipo-new-pypsa,2025-12-01 08:36:06.438968
7,pypsa-eur-sec-trex_vopt,50-3h,gurobi,12.0.3,2025.0,TO,Timeout,86400.000000,48765.440,NaN,NaN,NaN,86400.000000,86400.0,benchmark-instance-01-hipo-new-pypsa,20251201-hipo-new-pypsa,2025-12-01 08:19:46.293980
9,pypsa-eur-elec-trex_vopt,50-3h,gurobi,12.0.3,2025.0,ok,optimal,12590.782178,16082.528,3.532154e+10,NaN,NaN,12548.509729,86400.0,benchmark-instance-05-hipo-new-pypsa,20251201-hipo-new-pypsa,2025-12-01 08:18:34.272268
11,pypsa-eur-elec-trex_vopt,50-3h,highs-hipo-no2hop,1.12.0-hipo,2025.0,ok,Optimal,75187.224187,13072.440,3.532154e+10,NaN,NaN,75187.224187,86400.0,benchmark-instance-05-hipo-new-pypsa,20251201-hipo-new-pypsa,2025-12-01 11:53:08.144692
13,pypsa-eur-elec-trex_vopt,50-3h,highs-hipo-64,1.12.0-hipo,2025.0,TO,Timeout,86400.000000,164.716,NaN,NaN,NaN,86400.000000,86400.0,benchmark-instance-05-hipo-new-pypsa,20251201-hipo-new-pypsa,2025-12-02 08:49:55.443662
15,pypsa-eur-elec-trex_vopt,100-1h,gurobi,12.0.3,2025.0,OOM,Out of Memory,NaN,NaN,NaN,NaN,NaN,NaN,86400.0,benchmark-instance-06-hipo-new-pypsa,20251201-hipo-new-pypsa,2025-12-01 08:20:29.468980


Conclusion: most of these are too large! I killed the running VMs so as to not waste compute.

### 20251219 Run all sizes of a new PyPSA bench
Goal: to get a sense of which sizes are solvable

In [13]:
new_pypsa_benchs = load_benchmark_metadata("../benchmarks/pypsa/metadata.yaml")

In [14]:
# Run all de-elec instances > 5e5
to_run = new_pypsa_benchs.query(
    'Benchmark == "pypsa-de-elec" and `Num. variables` > 100000'
)
vm_yamls = allocate_benchmarks(
    to_run, "Num. variables", 11, machine_type="c4-highmem-8"
)  # 1 per VM

Allocated. Estimated runtime: 2399.4h
  VM 00: 1 instances, 2399.4h
  VM 01: 1 instances, 1070.7h
  VM 02: 1 instances, 799.9h
  VM 03: 1 instances, 586.5h
  VM 04: 1 instances, 356.9h
  VM 05: 1 instances, 200.1h
  VM 06: 1 instances, 195.5h
  VM 07: 1 instances, 100.1h
  VM 08: 1 instances, 89.3h
  VM 09: 1 instances, 48.9h
  VM 10: 1 instances, 44.7h


In [15]:
# Only run latest highs variants
for y in vm_yamls:
    y["years"] = [2025]
    y["solver"] = "highs-hipo highs-ipm highs"
create_benchmark_campaign("20251219-pypsa-sizes", "pypsa-sizes", vm_yamls)

Created directory and files in ../infrastructure/benchmarks/20251219-pypsa-sizes
Run this campaign from the infrastructure/ directory using the command:
tofu apply -var-file benchmarks/20251219-pypsa-sizes/run.tfvars -state=states/20251219-pypsa-sizes.tfstate
